In [17]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
clean_data_df = pd.read_csv("cleaned_data.csv",index_col="Unnamed: 0")
clean_data_df.head()

,enrollee_id,city_development_index,training_hours,target,gender_Female,gender_Male,gender_Other,relevent_experience_Has relevent experience,relevent_experience_No relevent experience,enrolled_university_Full time course,...,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,last_new_job_1,last_new_job_2,last_new_job_3,last_new_job_4,last_new_job_>4,last_new_job_never
1,29725,0.776,47,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,666,0.767,8,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
7,402,0.762,18,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
8,27107,0.920,46,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
11,23853,0.920,108,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [19]:
# define the  feaetures set
X=clean_data_df.copy()
X=X.drop("target",axis=1)
X.head()

,enrollee_id,city_development_index,training_hours,gender_Female,gender_Male,gender_Other,relevent_experience_Has relevent experience,relevent_experience_No relevent experience,enrolled_university_Full time course,enrolled_university_Part time course,...,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,last_new_job_1,last_new_job_2,last_new_job_3,last_new_job_4,last_new_job_>4,last_new_job_never
1,29725,0.776,47,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,666,0.767,8,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
7,402,0.762,18,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
8,27107,0.920,46,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
11,23853,0.920,108,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [20]:
# define target set
y=clean_data_df["target"].ravel()
y[:5]

array([0., 0., 1., 1., 0.])

In [21]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [22]:
#Create a StandardScaler instance
scaler=StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)
# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=500, random_state=78) 

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [24]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [25]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actually staying", "Actually leaving"], columns=["Predicted staying", "Predicted leaving"])

cm_df

,Predicted staying,Predicted leaving
Actually staying,842,25
Actually leaving,130,53


In [26]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

0.8523809523809524

In [27]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted staying,Predicted leaving
Actually staying,842,25
Actually leaving,130,53


Accuracy Score : 0.8523809523809524
Classification Report
              precision    recall  f1-score   support

         0.0       0.87      0.97      0.92       867
         1.0       0.68      0.29      0.41       183

    accuracy                           0.85      1050
   macro avg       0.77      0.63      0.66      1050
weighted avg       0.83      0.85      0.83      1050



In [28]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.1418682 , 0.25489472, 0.12715135, 0.00869312, 0.00938339,
       0.0017566 , 0.00887864, 0.0091052 , 0.00921118, 0.00763978,
       0.01074872, 0.01485392, 0.01481607, 0.0041703 , 0.00244453,
       0.00352576, 0.00679692, 0.00224307, 0.00398088, 0.00891414,
       0.00277909, 0.00870619, 0.00749891, 0.00729081, 0.00716676,
       0.00545226, 0.00836043, 0.00656325, 0.00799709, 0.00233003,
       0.00599497, 0.00586073, 0.00257726, 0.00729885, 0.00737479,
       0.00904812, 0.00908214, 0.00815639, 0.00631916, 0.00547825,
       0.00319859, 0.01550966, 0.0124034 , 0.01569778, 0.01262799,
       0.01462937, 0.01604077, 0.01010373, 0.00687147, 0.00773461,
       0.00577758, 0.00824093, 0.00741503, 0.00258133, 0.00643501,
       0.01317442, 0.01479748, 0.01071663, 0.00676572, 0.00694873,
       0.01291905, 0.00699875])

In [29]:
# We can sort the features by their importance.
x=zip(importances,X.columns)
sorted(x,reverse=True)

[(0.25489471741596637, 'city_development_index'),
 (0.14186819584385893, 'enrollee_id'),
 (0.12715134923714422, 'training_hours'),
 (0.016040770198212633, 'company_size_50-99'),
 (0.015697777995515857, 'company_size_100-500'),
 (0.015509664952601287, 'experience_>20'),
 (0.014853918765384926, 'education_level_Graduate'),
 (0.014816073641200335, 'education_level_Masters'),
 (0.014797481089987729, 'last_new_job_1'),
 (0.01462936794957455, 'company_size_10000+'),
 (0.013174422475192172, 'company_type_Pvt Ltd'),
 (0.012919052554280294, 'last_new_job_>4'),
 (0.012627992500325967, 'company_size_1000-4999'),
 (0.012403401595239968, 'company_size_10/49'),
 (0.010748718421376825, 'enrolled_university_no_enrollment'),
 (0.010716626547680078, 'last_new_job_2'),
 (0.01010373424015751, 'company_size_500-999'),
 (0.009383390279884133, 'gender_Male'),
 (0.009211183745331503, 'enrolled_university_Full time course'),
 (0.009105197136661098, 'relevent_experience_No relevent experience'),
 (0.00908213789